In [ ]:
import pickle
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.model_selection import train_test_split as splt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
dfy = pd.read_csv("MILANO_wSCORE.csv")
dfx = pd.read_csv("PLIC-milano-processed.csv", sep=";")

In [ ]:
df2 = dfx.replace(-1, np.nan).replace("-1", np.nan).replace(-1.0, np.nan)

In [ ]:
df3 = df2.dropna(how='any', axis=1)

In [ ]:
y = dfy['SCORE'].values

In [ ]:
df4 = df3.select_dtypes(exclude=['object'])

In [ ]:
df4 = df4.drop(labels = ['smoking', 'smoking recod', 'glucose', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [ ]:
X = df4.values

In [ ]:
bina = 0
for i in range(X.shape[1]):
    if (np.amax(X[:,i]) == 1):
        bina += 1
print(bina)

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=X.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()
X = X/X.max(axis=0)
y = y/50

In [ ]:
print(np.amax(X))
print(np.amax(y))
X_tr, X_ts, y_tr, y_ts = splt(X, y, test_size=0.25, random_state=42)
es = EarlyStopping(patience = 20, restore_best_weights = True)

In [ ]:
model.compile(loss = 'mean_squared_error', optimizer='sgd')

In [ ]:
history = model.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs = 2000, callbacks = [es])

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
model.save("model_MSE_SGD_2.h5")

In [ ]:
pickle.dump(history, open("history_MSE_SGD_2.pkl", "wb"))